In [ ]:
using HDF5
using PyPlot

In [ ]:
fname = "Model_Outputs/ModelRun09-27-2023_07:45:15/Amplitude_10.0%/Data.hdf5"
# Displaying all of the file's information
fid = h5open(fname, "r")

# Reading Data from file
modeldata = fid["Model Run"];

# Displaying the contents in Model Run group 
for obj in modeldata
    data = read(obj)
    println(obj)
    display(data)
end

# # Accessing Model Run contents that I want
Wavelength1 = read(modeldata,"Wavelength");
Ice_shell1 = read(modeldata, "Ice Shell Thickness");
# Time_halfspace = read(modeldata,"Viscous Relaxation Time(Half-Space)");
Time_rel1 = read(modeldata,"Viscous Relaxation Time(Model)");
Time_thickening1 = read(modeldata, "Thickening Time");

# Close file
close(fid)

In [ ]:
fname = "Model_Outputs/ModelRun09-27-2023_07:45:15/Amplitude_20.0%/Data.hdf5"
# Displaying all of the file's information
fid = h5open(fname, "r")

# Reading Data from file
modeldata = fid["Model Run"];

# Displaying the contents in Model Run group 
for obj in modeldata
    data = read(obj)
    println(obj)
    display(data)
end

# # Accessing Model Run contents that I want
Wavelength2 = read(modeldata,"Wavelength");
Ice_shell2 = read(modeldata, "Ice Shell Thickness");
# Time_halfspace = read(modeldata,"Viscous Relaxation Time(Half-Space)");
Time_rel2 = read(modeldata,"Viscous Relaxation Time(Model)");
Time_thickening2 = read(modeldata, "Thickening Time");

# Close file
close(fid)

In [ ]:
fname = "Model_Outputs/ModelRun09-27-2023_07:45:15/Amplitude_30.0%/Data.hdf5"
# Displaying all of the file's information
fid = h5open(fname, "r")

# Reading Data from file
modeldata = fid["Model Run"];

# Displaying the contents in Model Run group 
for obj in modeldata
    data = read(obj)
    println(obj)
    display(data)
end

# # Accessing Model Run contents that I want
Wavelength3 = read(modeldata,"Wavelength");
Ice_shell3 = read(modeldata, "Ice Shell Thickness");
# Time_halfspace = read(modeldata,"Viscous Relaxation Time(Half-Space)");
Time_rel3 = read(modeldata,"Viscous Relaxation Time(Model)");
Time_thickening3 = read(modeldata, "Thickening Time");

# Close file
close(fid)

In [ ]:
# println("Wavelength: $Wavelength")
# println("Ice Shell Thickness: $Ice_shell")
# println("Viscous Relaxation Time (Half-Space): $Time_halfspace")
# println("Viscous Relaxation Time (Model): $Time_rel")
# println("Thickening Time: $Time_thickening")

In [ ]:
 # is in time yrs 
# time/1e6 is Millions of Years
# time/1e3 is Thousands of Years

In [ ]:
figure()
cs = contour(Ice_shell1,Wavelength1,Time_rel1./Time_thickening1)
clabel(cs,inline=true,fmt="%0.1f        ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t'\,with\,10\%")
colorbar(label=L"t'",location="left")
gca().yaxis.set_label_position("right")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# gca().yaxis.tick_right()
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("right")
gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell2,Wavelength2,Time_rel2./Time_thickening2)
clabel(cs,inline=true,fmt="%0.1f        ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t'\,with\,20\%")
colorbar(label=L"t'",location="left")
gca().yaxis.set_label_position("right")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# gca().yaxis.tick_right()
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("right")
gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell3,Wavelength3,Time_rel3./Time_thickening3)
clabel(cs,inline=true,fmt="%0.1f        ",colors="w",fontsize=10)
contourf(Ice_shell3,Wavelength3,Time_rel3./Time_thickening3,cmap=get_cmap("viridis"))
title(L"t'\,with\,30\%")
colorbar(label=L"t'",location="left")
gca().yaxis.set_label_position("right")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# gca().yaxis.tick_right()
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("right")
gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

In [ ]:
figure()
cs = contour(Ice_shell1,Wavelength1,Time_rel1./1e6)
clabel(cs,inline=true,fmt="%0.1f         ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{relaxation}\,with\,10\%")
colorbar(label=L"t_{relaxation}",location="left")
gca().yaxis.set_label_position("right")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# gca().yaxis.tick_right()
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("right")
gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell2,Wavelength2,Time_rel2./1e6)
clabel(cs,inline=true,fmt="%0.1f         ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{relaxation}\,with\,20\%")
colorbar(label=L"t_{relaxation}",location="left")
gca().yaxis.set_label_position("right")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# gca().yaxis.tick_right()
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("right")
gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell3,Wavelength3,Time_rel3./1e6)
clabel(cs,inline=true,fmt="%0.1f         ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{relaxation}\,with\,30\%")
colorbar(label=L"t_{relaxation}",location="left")
gca().yaxis.set_label_position("right")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# gca().yaxis.tick_right()
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("right")
gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()


In [ ]:
# figure()
# cs = contour(Ice_shell1,Wavelength1,Time_thickening1./1e6)
# clabel(cs,inline=true,fmt="%0.1f         ",colors="w",fontsize=10)
# contourf(cs,cmap=get_cmap("viridis"))
# title(L"t_{thickening}\,with\,10\%")
# colorbar(label=L"t_{thickening}",location="left")
# gca().yaxis.set_label_position("right")
# gca().set_ylabel(L"\lambda\,(km)")
# gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# # gca().yaxis.set_label_coords(-0.23,0.5)
# gca().spines["top"].set_visible(false) # Hide the top edge of the axis
# gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# # gca().yaxis.tick_right()
# gca().xaxis.set_ticks_position("bottom")
# gca().yaxis.set_ticks_position("right")
# gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
# gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# # gca().spines["top"].set_color("none") # Remove the top axis boundary
# # gca().spines["right"].set_color("none") # Remove the right axis boundary
# # gca().set_xlim([10.0,30.0])
# # gca().set_ylim([10.0,100.0])
# tight_layout()
# # savefig("Trel_Ttic.jpeg",dpi=300)
# show()

# figure()
# cs = contour(Ice_shell2,Wavelength2,Time_thickening2./1e6)
# clabel(cs,inline=true,fmt="%0.1f         ",colors="w",fontsize=10)
# contourf(cs,cmap=get_cmap("viridis"))
# title(L"t_{thickening}\,with\,20\%")
# colorbar(label=L"t_{thickening}",location="left")
# gca().yaxis.set_label_position("right")
# gca().set_ylabel(L"\lambda\,(km)")
# gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# # gca().yaxis.set_label_coords(-0.23,0.5)
# gca().spines["top"].set_visible(false) # Hide the top edge of the axis
# gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# # gca().yaxis.tick_right()
# gca().xaxis.set_ticks_position("bottom")
# gca().yaxis.set_ticks_position("right")
# gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
# gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# # gca().spines["top"].set_color("none") # Remove the top axis boundary
# # gca().spines["right"].set_color("none") # Remove the right axis boundary
# # gca().set_xlim([10.0,30.0])
# # gca().set_ylim([10.0,100.0])
# tight_layout()
# # savefig("Trel_Ttic.jpeg",dpi=300)
# show()

# figure()
# cs = contour(Ice_shell3,Wavelength3,Time_thickening3./1e6)
# clabel(cs,inline=true,fmt="%0.1f         ",colors="w",fontsize=10)
# contourf(cs,cmap=get_cmap("viridis"))
# title(L"t_{thickening}\,with\,30\%")
# colorbar(label=L"t_{thickening}",location="left")
# gca().yaxis.set_label_position("right")
# gca().set_ylabel(L"\lambda\,(km)")
# gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# # gca().yaxis.set_label_coords(-0.23,0.5)
# gca().spines["top"].set_visible(false) # Hide the top edge of the axis
# gca().spines["left"].set_visible(false) # Hide the right edge of the axis
# # gca().yaxis.tick_right()
# gca().xaxis.set_ticks_position("bottom")
# gca().yaxis.set_ticks_position("right")
# gca().spines["right"].set_position(("axes",1.03)) # Offset the left scale from the axis
# gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# # gca().spines["top"].set_color("none") # Remove the top axis boundary
# # gca().spines["right"].set_color("none") # Remove the right axis boundary
# # gca().set_xlim([10.0,30.0])
# # gca().set_ylim([10.0,100.0])
# tight_layout()
# # savefig("Trel_Ttic.jpeg",dpi=300)
# show()

In [ ]:
# nx = 6
# ny = 6
# x = range(10.0,100.0,nx)
# y = range(10.0,100.0,ny)
# # z = zeros(nx,ny)
# f(x,y) = get_halfspace_time_viscous(x*1e3)/get_thickening_time(y*1e3)
# # z = @.f(x,y')

# # figure()
# # cs = contour(x,y,z)
# # clabel(cs,inline=true,colors="w",fontsize=10)
# # contourf(x,y,z)
# # colorbar(label=L"\frac{t_{half-space}}{t_{thickening}}")
# # title(L"\frac{t_{half-space}}{t_{thickening}}")
# # gca().set_xlabel(L"\lambda")
# # gca().set_ylabel(L"h_{ice\,shell}\,(km)",rotation=0)
# # # gca().set_xlim([0.0,0.2])
# # # gca().set_ylim([1.0,3.0])
# # gca().yaxis.set_label_coords(-0.23,0.5)
# # gca().spines["top"].set_visible(false) # Hide the top edge of the axis
# # gca().spines["right"].set_visible(false) # Hide the right edge of the axis
# # gca().xaxis.set_ticks_position("bottom")
# # gca().yaxis.set_ticks_position("left")
# # gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
# # gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# # show()
# # close() # Remove this line to show graph

# """ or """

# k = [f(i,j) for i in x, j in y]

# figure()
# cs = contour(x,y,k)
# clabel(cs,inline=true,colors="w",fontsize=10)
# contourf(x,y,k)
# title(L"\frac{t_{half-space}}{t_{thickening}}")
# colorbar(label=L"\frac{t_{half-space}}{t_{thickening}}")
# gca().set_xlabel(L"\lambda\,(km)")
# gca().set_ylabel(L"h_{ice\,shell}\,(km)",rotation=0)
# # gca().set_xlim([0.0,0.2])
# # gca().set_ylim([1.0,1.7])
# gca().yaxis.set_label_coords(-0.23,0.5)
# gca().spines["top"].set_visible(false) # Hide the top edge of the axis
# gca().spines["right"].set_visible(false) # Hide the right edge of the axis
# gca().xaxis.set_ticks_position("bottom")
# gca().yaxis.set_ticks_position("left")
# gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
# gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# show()

# figure()
# plot(transpose(x)./y,k,".-",label="21")
# gca().set_xlabel(L"\frac{\lambda}{h}")
# gca().set_ylabel(L"\frac{t_{half-space}}{t_{thickening}}",rotation=0)
# # gca().yaxis.set_label_coords(-0.2,0.5)
# gca().spines["top"].set_visible(false) # Hide the top edge of the axis
# gca().spines["right"].set_visible(false) # Hide the right edge of the axis
# gca().xaxis.set_ticks_position("bottom")
# gca().yaxis.set_ticks_position("left")
# gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
# gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# legend(loc="upper center", bbox_to_anchor=(0.5,-0.15),fancybox="True",shadow="True",ncol=5)
# show()